In [16]:
import pandas as pd
import numpy as np

dfRecipes = pd.read_csv('data/raw_recipes.csv')

# Shapes of dataframes
dfRecipes.rename(columns={'id':'recipe_id'}, inplace=True)
print("reviews:", dfRecipes.shape[0])

reviews: 231637


In [17]:
dfRecipes['IsMainDish'] = 0
dfRecipes.loc[dfRecipes['tags'].str.contains('\'main-dish\''), 'IsMainDish'] = 1

dfRecipes['IsMeatDish'] = 0
dfRecipes.loc[dfRecipes['tags'].str.contains('\'meat\''), 'IsMeatDish'] = 1

dfRecipes['IsVegetarian'] = 0
dfRecipes.loc[dfRecipes['tags'].str.contains('\'vegetarian\''), 'IsVegetarian'] = 1

dfRecipes['IsDessert'] = 0
dfRecipes.loc[dfRecipes['tags'].str.contains('\'desserts\''), 'IsDessert'] = 1

dfRecipes['IsHealthy'] = 0
dfRecipes.loc[dfRecipes['tags'].str.contains('\'healthy\''), 'IsHealthy'] = 1

dfRecipes['IsSideDish'] = 0
dfRecipes.loc[dfRecipes['tags'].str.contains('\'side-dishes\''), 'IsSideDish'] = 1

dfRecipes['HasEggsOrDairy'] = 0
dfRecipes.loc[dfRecipes['tags'].str.contains('\'eggs-dairy\''), 'HasEggsOrDairy'] = 1

dfRecipes['HasNuts'] = 0
dfRecipes.loc[dfRecipes['tags'].str.contains('\'nuts\''), 'HasNuts'] = 1

dfRecipes['HasShellfish'] = 0
dfRecipes.loc[dfRecipes['tags'].str.contains('\'shellfish\''), 'HasShellfish'] = 1

dfRecipes['IsLunch'] = 0
dfRecipes.loc[dfRecipes['tags'].str.contains('\'lunch\''), 'IsLunch'] = 1

dfRecipes['IsBreakfast'] = 0
dfRecipes.loc[dfRecipes['tags'].str.contains('\'breakfast\''), 'IsBreakfast'] = 1

In [18]:
# #Create fields for mealtime - and mealtype
# dfRecipes['MealTime'] = 'Any'
# dfRecipes.loc[dfRecipes['ingredients'].str.contains('\'side-dishes\''), 'MealTime'] = 'Side dish'
# dfRecipes.loc[dfRecipes['ingredients'].str.contains('\'desserts\''), 'MealTime'] = 'Desserts'
# dfRecipes.loc[dfRecipes['ingredients'].str.contains('\'breakfast\''), 'MealTime'] = 'Breakfast'
# dfRecipes.loc[dfRecipes['ingredients'].str.contains('\'lunch\''), 'MealTime'] = 'Lunch'
# dfRecipes.loc[dfRecipes['ingredients'].str.contains('\'main-dish\''), 'MealTime'] = 'Dinner'

# dfRecipes['MealType'] = 'Any'
# dfRecipes.loc[dfRecipes['ingredients'].str.contains('\'healthy\''), 'MealType'] = 'Healthy'
# dfRecipes.loc[dfRecipes['ingredients'].str.contains('\'meat\''), 'MealType'] = 'Meat'
# dfRecipes.loc[dfRecipes['ingredients'].str.contains('\'vegetarian\''), 'MealType'] = 'Vegetarian'

In [19]:
#remove some outliers
dfRecipes[['calories','fat','sugar','sodium','protein','saturated fat','carbs']] = dfRecipes['nutrition'].str.split(",",expand=True)
dfRecipes['calories'] = dfRecipes['calories'].str.strip('[').astype('float')
dfRecipes['carbs'] = dfRecipes['carbs'].str.strip(']').astype('float')
dfRecipes['fat'] = dfRecipes['fat'].astype('float')
dfRecipes['sugar'] = dfRecipes['sugar'].astype('float')
dfRecipes['sodium'] = dfRecipes['sodium'].astype('float')
dfRecipes['protein'] = dfRecipes['protein'].astype('float')
dfRecipes['saturated fat'] = dfRecipes['saturated fat'].astype('float')

topQuantileSteps = dfRecipes['n_steps'].quantile(0.95)
print('Number of steps in the top 95 quantile', topQuantileSteps)

topQuantileIngredients = dfRecipes['n_ingredients'].quantile(0.95)
print('Number of ingredients in the top 95 quantile', topQuantileIngredients)

topQuantileCalories = dfRecipes['calories'].quantile(0.95)
print('Number of calories in the top 95 quantile', topQuantileCalories)

topQuantileMinutes = dfRecipes['minutes'].quantile(0.95)
print('Number of minutes in the top 95 quantile', topQuantileMinutes)

Number of steps in the top 95 quantile 21.0
Number of ingredients in the top 95 quantile 16.0
Number of calories in the top 95 quantile 1268.7199999999982
Number of minutes in the top 95 quantile 255.0


In [20]:
#Remove the outliers
dfRecipesDropped = dfRecipes.loc[(dfRecipes['calories'] <= topQuantileCalories) & (dfRecipes['n_ingredients'] <= topQuantileIngredients) & (dfRecipes['n_steps'] <= topQuantileSteps) & (dfRecipes['minutes'] <= topQuantileMinutes)]

print('Reduced to',dfRecipesDropped.shape[0],'rows')

Reduced to 195442 rows


In [21]:
#Get the review data
dfReviews = pd.read_csv('data/raw_interactions.csv')
print(dfReviews.shape[0])

1132367


In [22]:
dfReviewsGroup = dfReviews.groupby('recipe_id')['rating'].agg(['sum','count']).sort_values(by='count',ascending=False)

dfReviewsGroup['rating_average']= dfReviewsGroup['sum'] / dfReviewsGroup['count']
dfReviewsGroup.rename(columns={'sum':'rating_sum'}, inplace=True)
dfReviewsGroup.rename(columns={'count':'rating_count'}, inplace=True)
print('Recipes Reviews Grouped Row Count', dfReviewsGroup.shape[0])
print(dfReviewsGroup.head(10))

Recipes Reviews Grouped Row Count 231637
           rating_sum  rating_count  rating_average
recipe_id                                          
2886             6752          1613        4.185989
27208            6866          1601        4.288570
89204            6664          1579        4.220393
39087            6576          1448        4.541436
67256            5723          1322        4.329047
54257            5499          1305        4.213793
22782            5458          1234        4.423015
32204            5521          1220        4.525410
69173            4694           997        4.708124
68955            3976           904        4.398230


In [23]:
avgReview = dfReviewsGroup['rating_average'].mean()
print('Average Review score is', avgReview)

Average Review score is 4.346246073856186


In [24]:
topQuantile = dfReviewsGroup['rating_count'].quantile(0.99)
print('Number of votes in the top 99 quantile', topQuantile)

Number of votes in the top 99 quantile 47.0


In [25]:
topQuantileReviews = dfReviewsGroup.copy().loc[dfReviewsGroup['rating_count'] >= topQuantile]
topQuantileReviews

,rating_sum,rating_count,rating_average
recipe_id,,,
2886,6752,1613,4.185989
27208,6866,1601,4.288570
89204,6664,1579,4.220393
39087,6576,1448,4.541436
67256,5723,1322,4.329047
...,...,...,...
76453,211,47,4.489362
69850,209,47,4.446809
40107,215,47,4.574468


In [26]:
def weighted_rating(x, m=topQuantile, C=avgReview):
    v = x['rating_count']
    R = x['rating_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [27]:
topQuantileReviews['weightedRating'] = topQuantileReviews.apply(weighted_rating, axis=1)

In [28]:
dfRecipes = dfRecipes.merge(dfReviewsGroup, left_on="recipe_id", right_on="recipe_id", how="inner")
#dfRecipes.head(5)

In [32]:
#Finally save all the data
dfReduced = dfRecipes.sample(frac=.1)
dfReduced.to_csv('data/reducedRecipes.csv', index=False)
print('Sample 10% reduces the dataframe to',dfReduced.shape[0],'rows')

Sample 10% reduces the dataframe to 23164 rows
